### Import packages

In [1]:
import urllib.request
import re 
import numpy as np
import pandas as pd
import time
import random
from bs4 import BeautifulSoup


### Initialisation des paramètres

In [72]:
annees = ['2023'] #'2019', '2020', '2021', '2022', '2023'
type_competition = ['Cross'] # Salle', 'Hors+Stade', 'Cross', 'Stade'
niveaux = ['D%C3%A9partemental','R%C3%A9gional', 'Interr%C3%A9gional', 'National', 'International', 'Europ%C3%A9en', 'Mondial'] #

# %C3%A9

### Récupération des liens des compétitions

In [73]:
links_total=[]

# On veut les résultats de toutes les compétitions, de tous niveaux et pour chaque année étudiée
# Dans un premier temps on récupère tous les liens de ces compétitions

for annee1 in annees:

    for compet in type_competition: 
        
        
        for niveau in niveaux: 
            
            url1="https://bases.athle.fr/asp.net/liste.aspx?frmpostback=true&frmbase=resultats&frmmode=2&frmespace=0&frmsaison=" + str(annee1) + "&frmtype1=" + str(compet) + "&frmtype2=&frmtype3=&frmtype4=&frmniveau=" + str(niveau) + "&frmniveaulab=&frmligue=&frmdepartement=&frmeprrch=&frmclub=&frmdate_j1=&frmdate_m1=&frmdate_a1=&frmdate_j2=&frmdate_m2=&frmdate_a2="

            try: 
                
                soup1= BeautifulSoup(urllib.request.urlopen(url1, timeout=15), 'html.parser')

                count = len(soup1.find_all('select', class_='barSelect'))
                pages=[]
                    
                if count > 0: 
                    options2 = soup1.find_all('select', class_='barSelect')[0].find_all('option')
                    pages = [option2['value'] for option2 in options2 if option2['value'] != '']

                else: 
                    pages.append('')
                        
                # on cherche dans chaque page de résultat pour chaque type de compétition pour une année donnée
                        
                for p in pages: 
                            
                    url2 = url1 + '&frmposition=' + str(p)
                    time.sleep(0.5)
                    soup2 = BeautifulSoup(urllib.request.urlopen(url2, timeout=15), 'html.parser')

                    links=[]

                    for link in soup2.findAll('a', {'class':''})[1::2]:
                        link = link.get('href')
                        link = 'https://bases.athle.fr'+str(link)
                        if "podiums" not in link:
                            links.append(str(link))
                        
                        
                        
                    links_total = links_total + links
                        
                    sleep_time = random.uniform(2,4)
                    time.sleep(sleep_time)  
            
            except: 
                continue  
                        




In [10]:
links_total = pd.read_excel("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Liens compétitions/liens_cross_2020.xlsx", index_col=0)
links_total = links_total.iloc[:,0].tolist()


### Récupération des infos sur la compétition et les résultats

In [11]:
resultats = pd.DataFrame()
        
# Pour chaque url de compétition ainsi récupéré            

for l in links_total: 
    
    url3 = l
    
    #sleep_time = random.uniform(2,4)
    #time.sleep(sleep_time) 
    
    try:
        
    # on l'ouvre
        soup3 = BeautifulSoup(urllib.request.urlopen(url3, timeout=15), 'html.parser')
        
        # On récupère les infos sur la compétition
        
        # Extraction de la date et du nom de la compétition

        first_part = soup3.find('div', class_='mainheaders').text.split('<br/>')[0].strip()
        date, nom_competition_part = first_part.split(' - ', 1)
        nom_competition = nom_competition_part.split(' - ')[0].strip()

    
    # Extraction du lieu, de la region et du département
        span_text = soup3.find('div', class_='mainheaders').find('span').text.split(' - ')

        if len(span_text)==1: 
            span_text.append("Region etrangere")
            span_text.append("Departement etranger")
        if len(span_text) < 4: 
            span_text.append("Pas de label")
        
        lieu_competition, region_competition, departement_competition, label_competition = map(str.strip, span_text)

        nom_competition = nom_competition.replace(lieu_competition,'').strip()
        
        if "kid" not in nom_competition.lower() and "lancer" not in nom_competition.lower() and "animation" not in nom_competition.lower() and "disque" not in nom_competition.lower() and "combinées" not in nom_competition.lower():
        
            # on récupère les différentes épreuves qui composent la compétition
            
            epreuves = []
            
            count2 = len(soup3.find_all('select', class_='barSelect'))
            options = soup3.find_all('select', class_='barSelect')[int(count2/2-1)].find_all('option')
            
            epreuves = [option['value'] for option in options if option['value'] != '']
            
            # pour chacune de ces épreuves
            
            
            for epreuve in epreuves: 
                
                
                # on définit l'url qui lui est associé
                url4 = url3 + '&frmepreuve=' + str(epreuve)
                
            
                
                #time.sleep(0.5) 
            
                # on l'ouvre
                soup4 = BeautifulSoup(urllib.request.urlopen(url4, timeout=15), 'html.parser')
                
                # on récupère les infos sur l'épreuve
                
                epreuve = soup4.find('div', class_='subheaders').text.split(' | ')[0]
                
                if len(soup4.find('div', class_='subheaders').text.split(' | ')) > 4:
                    heure_epreuve = re.sub(r'[^0-9:]', '', soup4.find('div', class_='subheaders').text.split(' | ')[-1])
                
                else: 
                    heure_epreuve = "Pas d'horaire"
                
                
                # on récupère le nombre de pages de résultat de l'épreuve
                
                count3 = len(soup4.find_all('select', class_='barSelect'))
                pages=[]
                
                if count3 > 6: 
                    options3 = soup4.find_all('select', class_='barSelect')[0].find_all('option')
                    pages = [option3['value'] for option3 in options3 if option3['value'] != '']
                else: 
                    pages.append('')
                
                # on cherche dans chaque page de résultat pour chaque épreuve
                    
                for p in pages: 
                    
                    url5 = url4 + '&frmposition=' + str(p)
                    
                    #time.sleep(0.5) 
                    
                    soup5 = BeautifulSoup(urllib.request.urlopen(url5, timeout=15), 'html.parser')
                
                    # récupération des infos sur les résultats des athlètes et la compétition sous forme de liste
            
                    perf = []
                    annee_naissance = []
                    nom_athlete = []
                    categorie = []
                    club = []
                    ligue = []
                    departement = []
                    position = []
                    
                    
                    for i in soup5.select('tr td.datas0:first-child, tr td.datas1:first-child'):
                        position_i = i.get_text()
                        if re.match("^[0-9'\"]*$", position_i):
                            position.append(str(position_i))
                    for i in soup5.select('tr td.datas0:nth-child(3), tr td.datas1:nth-child(3)'):
                        perf_i = i.get_text()
                        if len(perf_i)<20:
                            perf.append(str(perf_i))
                    for i in soup5.select('tr td.datas0:nth-child(5), tr td.datas1:nth-child(5)'):
                        nom_athlete_i = i.get_text()
                        nom_athlete.append(str(nom_athlete_i))
                    for i in soup5.select('tr td.datas0:nth-child(7), tr td.datas1:nth-child(7)'):
                        club_i = i.get_text()
                        club.append(str(club_i))
                    for i in soup5.select('tr td.datas0:nth-child(9), tr td.datas1:nth-child(9)'):
                        departement_i = i.get_text()
                        departement.append(str(departement_i))
                    for i in soup5.select('tr td.datas0:nth-child(11), tr td.datas1:nth-child(11)'):
                        ligue_i = i.get_text()
                        ligue.append(str(ligue_i))
                    for i in soup5.select('tr td.datas0:nth-child(13), tr td.datas1:nth-child(13)'):
                        categorie_i = i.get_text()
                        cat=categorie_i.split('/')[0]
                        annee=categorie_i.split('/')[1]
                        categorie.append(str(cat))
                        annee_naissance.append(str(annee))
                        
                    
                        
                    
                    
                    # transformation des listes en dataframe
                    
                    donnees = {'Position': position,
                                'Nom_athlete': nom_athlete,
                                'Performance': perf,
                                'Annee_naissance': annee_naissance, 
                                'Categorie': categorie,
                                'Club': club,
                                'Département': departement,
                                'Ligue': ligue, 
                                'Epreuve': epreuve,
                                'Nom_competition': nom_competition,
                                'Lieu_competition': lieu_competition,
                                'Date_competition': date,
                                'Heure_epreuve': heure_epreuve,
                                'Departement_competition': departement_competition,
                                'Region_competition': region_competition,
                                'Label_competition': label_competition}
                    
                    
                    
                    
                    resultats_partiels = pd.DataFrame(donnees)
                    resultats = pd.concat([resultats, resultats_partiels], ignore_index=True)
            
    except: 
        continue




In [12]:
resultats.to_excel(f'Resultats_cross_2020.xlsx') #_{compet}_{annee1}

In [19]:
## Test récupération des données pour une compétition donnée
# 
#
resultats = pd.DataFrame()

url2 = "https://bases.athle.fr/asp.net/liste.aspx?frmbase=resultats&frmmode=1&frmespace=0&frmcompetition=275068"
    
    # on l'ouvre
soup2 = BeautifulSoup(urllib.request.urlopen(url2, timeout=15), 'html.parser')
    
    # On récupère les infos sur la compétition
    
    # Extraction de la date

first_part = soup2.find('div', class_='mainheaders').text.split('<br/>')[0].strip()
date, nom_competition_part = first_part.split(' - ', 1)
nom_competition = nom_competition_part.split(' - ')[0].strip()


    # Extraction du lieu, de la region et du département
span_text = soup2.find('div', class_='mainheaders').find('span').text.split('-')

if len(span_text)==1: 
    span_text.append("Region etrangere")
    span_text.append("Departement etranger")
if len(span_text) < 4: 
    span_text.append("Pas de label")
    
lieu_competition, region_competition, departement_competition, label_competition = map(str.strip, span_text)

nom_competition = nom_competition.replace(lieu_competition,'').strip()
    
    # on récupère les différentes épreuves qui composent la compétition
    
epreuves = []
    
count = len(soup2.find_all('select', class_='barSelect'))
options = soup2.find_all('select', class_='barSelect')[int(count/2-1)].find_all('option')
    
epreuves = [option['value'] for option in options if option['value'] != '']
    
for epreuve in epreuves: 
        
        # on définit l'url qui lui est associé
    url3 = url2 + '&frmepreuve=' + str(epreuve)        
        # on définit l'url qui lui est associé
        # on l'ouvre
    soup3 = BeautifulSoup(urllib.request.urlopen(url3, timeout=15), 'html.parser')
        
        # on récupère les infos sur l'épreuve
        
    epreuve = soup3.find('div', class_='subheaders').text.split(' | ')[0]
        
    if len(soup3.find('div', class_='subheaders').text.split(' | ')) > 4:
        heure_epreuve = re.sub(r'[^0-9:]', '', soup3.find('div', class_='subheaders').text.split(' | ')[-1])
        
    else: 
        heure_epreuve = "Pas d'horaire"
        
        
        # on récupère le nombre de pages de résultat de l'épreuve
        
    count2 = len(soup3.find_all('select', class_='barSelect'))
    pages=[]
        
    if count2 > 6: 
        options2 = soup3.find_all('select', class_='barSelect')[0].find_all('option')
        pages = [option2['value'] for option2 in options2 if option2['value'] != '']

    else: 
        pages.append('')
        
# on cherche dans chaque page de résultat pour chaque épreuve
            
    for p in pages: 
            
        url4 = url3 + '&frmposition=' + str(p)
            
        soup4 = BeautifulSoup(urllib.request.urlopen(url4, timeout=15), 'html.parser')
        
            # récupération des infos sur les résultats des athlètes et la compétition sous forme de liste
    
        perf = []
        annee_naissance = []
        nom_athlete = []
        categorie = []
        club = []
        ligue = []
        departement = []
        position = []
            
            
        for i in soup4.select('tr td.datas0:first-child, tr td.datas1:first-child'):
            position_i = i.get_text()
            position.append(str(position_i))
        for i in soup4.select('tr td.datas0:nth-child(3), tr td.datas1:nth-child(3)'):
            perf_i = i.get_text()
            perf.append(str(perf_i))
        for i in soup4.select('tr td.datas0:nth-child(5), tr td.datas1:nth-child(5)'):
            nom_athlete_i = i.get_text()
            nom_athlete.append(str(nom_athlete_i))
        for i in soup4.select('tr td.datas0:nth-child(7), tr td.datas1:nth-child(7)'):
            club_i = i.get_text()
            club.append(str(club_i))
        for i in soup4.select('tr td.datas0:nth-child(9), tr td.datas1:nth-child(9)'):
            departement_i = i.get_text()
            departement.append(str(departement_i))
        for i in soup4.select('tr td.datas0:nth-child(11), tr td.datas1:nth-child(11)'):
            ligue_i = i.get_text()
            ligue.append(str(ligue_i))
        for i in soup4.select('tr td.datas0:nth-child(13), tr td.datas1:nth-child(13)'):
            categorie_i = i.get_text()
            cat=categorie_i.split('/')[0]
            annee=categorie_i.split('/')[1]
            categorie.append(str(cat))
            annee_naissance.append(str(annee))
                
            
                
            
            
            # transformation des listes en dataframe
            
        donnees = {'Position': position,
                         'Nom_athlete': nom_athlete,
                         'Performance': perf,
                         'Annee_naissance': annee_naissance, 
                         'Categorie': categorie,
                         'Club': club,
                         'Département': departement,
                         'Ligue': ligue, 
                         'Epreuve': epreuve,
                         'Nom_competition': nom_competition,
                         'Lieu_competition': lieu_competition,
                         'Date_competition': date,
                         'Heure_epreuve': heure_epreuve,
                         'Departement_competition': departement_competition,
                         'Region_competition': region_competition,
                         'Label_competition': label_competition
                         }
    
        resultats_partiels = pd.DataFrame(donnees)
            
        resultats = pd.concat([resultats, resultats_partiels], ignore_index=True)  
            
          

In [13]:
## Test prise en compte du numéro de page

url='https://bases.athle.fr/asp.net/liste.aspx?frmpostback=true&frmbase=resultats&frmmode=2&frmespace=0&frmsaison=2023&frmtype1=Salle&frmtype2=&frmtype3=&frmtype4=&frmniveau=Interr%C3%A9gional&frmniveaulab=&frmligue=&frmdepartement=&frmeprrch=&frmclub=&frmdate_j1=&frmdate_m1=&frmdate_a1=&frmdate_j2=&frmdate_m2=&frmdate_a2='

soup = BeautifulSoup(urllib.request.urlopen(url, timeout=15), 'html.parser')

count = len(soup.find_all('select', class_='barSelect'))
print(count)

pages=[]
        
if count > 0: 
     options2 = soup.find_all('select', class_='barSelect')[0].find_all('option')
     pages = [option2['value'] for option2 in options2 if option2['value'] != '']

else: 
     pages.append('')
            
            


# Afficher les valeurs extraites


print(url + '&frmposition=' + pages[0])


pages

0
https://bases.athle.fr/asp.net/liste.aspx?frmpostback=true&frmbase=resultats&frmmode=2&frmespace=0&frmsaison=2023&frmtype1=Salle&frmtype2=&frmtype3=&frmtype4=&frmniveau=Interr%C3%A9gional&frmniveaulab=&frmligue=&frmdepartement=&frmeprrch=&frmclub=&frmdate_j1=&frmdate_m1=&frmdate_a1=&frmdate_j2=&frmdate_m2=&frmdate_a2=&frmposition=


['']

In [80]:
## Test récupération infos sur la compétition


url = "https://bases.athle.fr/asp.net/liste.aspx?frmbase=resultats&frmmode=1&frmespace=0&frmcompetition=279052"
soup = BeautifulSoup(urllib.request.urlopen(url, timeout=15), 'html.parser')

# Extraction de la date

first_part = soup.find('div', class_='mainheaders').text.split('<br/>')[0].strip()
date, nom_competition_part = first_part.split(' - ', 1)
nom_competition = nom_competition_part.split(' - ')[0].strip()



# Extraction du lieu, de la region et du département
span_text = soup.find('div', class_='mainheaders').find('span').text.split('-')

if len(span_text) < 4: 
    span_text.append("Pas de label")
    
lieu_competition, region_competition, departement_competition, label_competition = map(str.strip, span_text)

nom_competition = nom_competition.replace(lieu_competition,'').strip()


print(span_text)
print(first_part)

# Affichage des résultats
print(f'Date: {date}')
print(f'Nom de la compétition: {nom_competition}')
print(f'Lieu de la compétition: {lieu_competition}')
print(f'Région: {region_competition}')
print(f'Département: {departement_competition}')
print(label_competition)

date = [date] * 10
print(date)

['Riorges ', ' ARA ', ' 042 ', ' Label Régional']
05/11/23 - 10 km de RiorgesRiorges - ARA - 042 - Label Régional
Date: 05/11/23
Nom de la compétition: 10 km de
Lieu de la compétition: Riorges
Région: ARA
Département: 042
Label Régional
['05/11/23', '05/11/23', '05/11/23', '05/11/23', '05/11/23', '05/11/23', '05/11/23', '05/11/23', '05/11/23', '05/11/23']


In [76]:
## Test récuération infos sur l'épreuve

url = "https://bases.athle.fr/asp.net/liste.aspx?frmbase=resultats&frmmode=1&frmespace=0&frmcompetition=275068&frmepreuve=U16+masculin+MIM"
soup = BeautifulSoup(urllib.request.urlopen(url, timeout=15), 'html.parser')

epreuve = soup.find('div', class_='subheaders').text.split(' | ')[2]
print(epreuve)
distance = soup.find('div', class_='subheaders').text.split(' | ')[4]
print(distance)
heure_competition = re.sub(r'[^0-9:]', '', soup.find('div', class_='subheaders').text.split(' | ')[6])
print(heure_competition)


U16 masculin MIM
4510 m
11:05
